In [19]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

In [20]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [21]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

In [22]:
def calculate_EAR(eye_landmarks):
    A = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    B = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    C = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

In [23]:
def estimate_head_direction(landmarks):
    nose_tip = np.array(landmarks[1])
    left_eye = np.mean(landmarks[2:4], axis=0)
    right_eye = np.mean(landmarks[5:7], axis=0)
    face_vector = right_eye - left_eye
    direction = 'Center'
    if face_vector[0] > 0:
        direction = 'Right'
    elif face_vector[0] < 0:
        direction = 'Left'
    return direction

In [24]:
def is_looking_down(landmarks):
    nose_tip = np.array(landmarks[1])
    chin = np.array(landmarks[152])  # 턱의 랜드마크
    vertical_distance = nose_tip[1] - chin[1]
    
    if vertical_distance < 20:  # 고개가 많이 숙여졌다면
        return True
    return False

In [25]:
def is_looking_at_phone(landmarks):
    left_eye = [landmarks[i] for i in [362, 385, 387, 263, 373, 380]]
    right_eye = [landmarks[i] for i in [33, 160, 158, 133, 153, 144]]
    
    eye_center = np.mean([landmarks[468], landmarks[473]], axis=0)  # 눈동자 중심
    eye_bottom = (left_eye[2][1] + right_eye[2][1]) / 2  # 눈 아래쪽 랜드마크의 Y좌표
    
    if eye_center[1] > eye_bottom:  # 눈동자가 아래쪽을 보고 있는 경우
        return True
    return False

In [26]:
# 눈 깜빡임 및 졸음 감지 파라미터
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 48
blink_counter = 0
drowsy = False

In [27]:
cap = cv2.VideoCapture(0)

In [29]:
# Mediapipe face mesh 객체 생성
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    while cap.isOpened():
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 프레임이 비어 있는지 확인
        if not ret or frame is None:
            print("프레임을 읽을 수 없습니다.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            rgb_frame = cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_frame)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    height, width, _ = roi_color.shape
                    landmarks = [(int(pt.x * width), int(pt.y * height)) for pt in face_landmarks.landmark]

                    left_eye_landmarks = [landmarks[i] for i in [362, 385, 387, 263, 373, 380]]
                    right_eye_landmarks = [landmarks[i] for i in [33, 160, 158, 133, 153, 144]]

                    left_EAR = calculate_EAR(left_eye_landmarks)
                    right_EAR = calculate_EAR(right_eye_landmarks)
                    EAR = (left_EAR + right_EAR) / 2.0

                    face_direction = estimate_head_direction(landmarks)

                    if EAR < EYE_AR_THRESH:
                        blink_counter += 1
                    else:
                        if blink_counter >= EYE_AR_CONSEC_FRAMES:
                            drowsy = True
                        blink_counter = 0

                    if is_looking_down(landmarks) and is_looking_at_phone(landmarks):
                        cv2.putText(frame, "넌 지금 휴대폰을 보고 있다.", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    elif drowsy:
                        cv2.putText(frame, "다 졸았니?", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    mp_drawing.draw_landmarks(roi_color, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

        cv2.imshow('Eyee Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()